# Portfolio Analysis

In this project, we seek to run through some classic and modern portfolio construction and asset allocation strategies like Modern Portfolio Theory (Markovitz, 1952), Black-Litterman Model (Black & Litterman, 1992) , Hierarchical Risk Parity (Lopez de Prado, 2016) etc. to gain some insights into the usefulness of standard risk measures like sharpe ratio and maximum drawdown in constructing a risk-adjusted portfolio. The end goal is to test such strategies to a sufficient level of rigour that we can convincingly employ them into our portfolio rebalancing pipeline. I will start off by testing my own portfolio and the procedures should be generalizable. Let's get started! 

(**Note**: *Some of the inputs used are based off my personal fundamental analysis so they are not yet documented.*)

## 0. Project Setup

##### Assets

1. Sea Limited (NYSE:'**SE**')
2. Unity Software Inc (NYSE:'**U**')
3. NVIDIA Corporation (NASDAQ:'**NVDA**') 
4. Bilibili Inc (NASDAQ:'**BILI**')
5. Facebook Inc (NASDAQ:'**FB**')
6. Draftkings Inc (NASDAQ:'**DKNG**')
7. Pinduoduo Inc (NASDAQ:'**PDD**')
8. Alphabet Inc (NASDAQ:'**GOOG**)
9. Amazon.com Inc (NASDAQ:'**AMZN**)
10. Roblox Corp (NYSE:'**RBLX**')
11. Clearpoint Neuro Inc(NASDAQ: '**CLPT**')
12. Intellia Therapeutics Inc (NASDAQ: '**NTLA**')
13. Coinbase Global Inc (NASDAQ: '**COIN**')
14. Upstart Holdings Inc (NASDAQ: '**UPST**')

##### Data

Daily adjusted closing prices from yahoo finance API 

##### Time period

1 year, 3 year, 5 year & 10 year.

In [2]:
# Load the required packages 
# Computation
import numpy as np 
from scipy import fftpack
# Plotting
import matplotlib.pyplot as plt
import mplcursors
import matplotlib.ticker as mtick
import seaborn as sns
# Data analysis
import pandas as pd
from sklearn import preprocessing
# Data source
import yfinance as yf

## 1. Data Analysis

In [48]:
# shortlisted stocks for portfolio analysis
s_list = 'SE U NVDA BILI FB DKNG PDD GOOG AMZN RBLX NTLA COIN UPST CLPT'
df_1y = yf.download(tickers = s_list, period = '1y', interval = '1d', group_by = 'ticker')
df_3y = yf.download(tickers = s_list, period = '3y', interval = '1d', group_by = 'ticker')
df_5y = yf.download(tickers = s_list, period = '5y', interval = '1d', group_by = 'ticker')
df_10y = yf.download(tickers = s_list, period = '10y', interval = '1d', group_by = 'ticker')

# Benchmark - S&P500
bench = '^GSPC'
bench_1y = yf.download(tickers = bench, period = '1y', interval = '1d')
bench_3y = yf.download(tickers = bench, period = '3y', interval = '1d')
bench_5y = yf.download(tickers = bench, period = '5y', interval = '1d')
bench_10y = yf.download(tickers = bench, period = '10y', interval = '1d')

[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [49]:
# select column for adjusted close prices
df_close_1y = df_1y.xs('Adj Close', level = 1, axis = 1)
df_close_3y = df_3y.xs('Adj Close', level = 1, axis = 1)
df_close_5y = df_5y.xs('Adj Close', level = 1, axis = 1)
df_close_10y = df_10y.xs('Adj Close', level = 1, axis = 1)
bench_close_1y = pd.DataFrame(data = bench_1y['Adj Close'], columns = ['Adj Close'])
bench_close_3y = pd.DataFrame(data = bench_3y['Adj Close'], columns = ['Adj Close'])
bench_close_5y = pd.DataFrame(data = bench_5y['Adj Close'], columns = ['Adj Close'])
bench_close_10y = pd.DataFrame(data = bench_10y['Adj Close'], columns = ['Adj Close'])

# adding column name to benchmark table
bench_close_1y = bench_close_1y.rename(columns = {'Adj Close':'S&P500'})
bench_close_3y = bench_close_3y.rename(columns = {'Adj Close':'S&P500'})
bench_close_5y = bench_close_5y.rename(columns = {'Adj Close':'S&P500'})
bench_close_10y = bench_close_10y.rename(columns = {'Adj Close':'S&P500'})
 
# reordering column names in portfolio 
s_order = ['SE', 'U', 'NVDA', 'BILI', 'FB', 'DKNG', 'PDD', 'GOOG', 'AMZN', 'RBLX', 'NTLA', 'COIN', 'UPST', 'CLPT']
df_close_1y = df_close_1y[s_order]
df_close_3y = df_close_3y[s_order]
df_close_5y = df_close_5y[s_order]
df_close_10y = df_close_10y[s_order]

# appending benchmark prices to portfolio dataframe 
df_close_1y = pd.concat([df_close_1y,bench_close_1y], axis = 1)
df_close_3y = pd.concat([df_close_3y,bench_close_3y], axis = 1)
df_close_5y = pd.concat([df_close_5y,bench_close_5y], axis = 1)
df_close_10y = pd.concat([df_close_10y,bench_close_10y], axis = 1)

# drop row if all values are NaN
df_close_1y.dropna(axis = 0, how = 'all', inplace = True)
df_close_3y.dropna(axis = 0, how = 'all', inplace = True)
df_close_5y.dropna(axis = 0, how = 'all', inplace = True)
df_close_10y.dropna(axis = 0, how = 'all', inplace = True)

# check the last 5 trading days
df_close_1y.tail(5)

,SE,U,NVDA,BILI,FB,DKNG,PDD,GOOG,AMZN,RBLX,NTLA,COIN,UPST,CLPT,S&P500
Date,,,,,,,,,,,,,,,
2021-10-01,319.559998,126.220001,207.419998,64.879997,343.010010,50.560001,88.150002,2729.250000,3283.260010,75.589996,134.059998,231.149994,298.769989,17.889999,4357.040039
2021-10-04,317.899994,119.849998,197.320007,61.990002,326.230011,48.459999,85.080002,2675.300049,3189.780029,77.800003,126.779999,229.309998,289.769989,17.000000,4300.459961
2021-10-05,320.940002,128.050003,204.509995,64.500000,332.959991,49.290001,87.419998,2723.540039,3221.000000,72.570000,129.759995,240.089996,296.299988,17.150000,4345.720215
2021-10-06,315.339996,130.380005,207.000000,64.269997,333.640015,48.990002,89.320000,2747.080078,3262.010010,73.699997,129.690002,250.380005,307.700012,17.280001,4363.549805
2021-10-07,324.890015,136.160004,211.840393,70.949997,333.654999,50.075001,94.709999,2796.649902,3318.449951,75.349998,134.335007,251.337906,312.747009,17.490000,4417.069824


In [50]:
# normalize the prices 
n = len(df_close_1y.columns)
df_close_1y_norm = df_close_1y.copy()
df_close_3y_norm = df_close_3y.copy()
df_close_5y_norm = df_close_5y.copy()
df_close_10y_norm = df_close_10y.copy()

for i in range(n): 
    a = df_close_1y.iloc[:,i].first_valid_index()
    df_close_1y_start = df_close_1y.iloc[:,i].loc[a]
    df_close_1y_norm.iloc[:,i] = (df_close_1y.iloc[:,i] - df_close_1y_start)/df_close_1y_start
    
    b = df_close_3y.iloc[:,i].first_valid_index()
    df_close_3y_start = df_close_3y.iloc[:,i].loc[b]
    df_close_3y_norm.iloc[:,i] = (df_close_3y.iloc[:,i] - df_close_3y_start)/df_close_3y_start
    
    c = df_close_5y.iloc[:,i].first_valid_index()
    df_close_5y_start = df_close_5y.iloc[:,i].loc[c]
    df_close_5y_norm.iloc[:,i] = (df_close_5y.iloc[:,i] - df_close_5y_start)/df_close_5y_start
    
    d = df_close_10y.iloc[:,i].first_valid_index()
    df_close_10y_start = df_close_10y.iloc[:,i].loc[d]
    df_close_10y_norm.iloc[:,i] = (df_close_10y.iloc[:,i] - df_close_10y_start)/df_close_10y_start

In [51]:
# plotting out the prices
%matplotlib widget

# plot configurations
sns.set(style="darkgrid", font_scale=0.8)
palette = sns.color_palette("hls", n)
fig, ax = plt.subplots(figsize=(8, 4))

# plotting out the figure
plot1 = sns.lineplot(ax=ax, data = df_close_1y_norm, dashes = False, palette=palette)
plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0., fontsize = 8)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.title('1 Year Returns')

# make it interactive
cursor = mplcursors.cursor(plot1, hover=True)
@cursor.connect("add")
def on_add(sel):
    sel.annotation.set(text=tt[sel.target.index])
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
# convert the price into rate of return
df_return_1y = df_close_1y.pct_change()
df_return_3y = df_close_3y.pct_change()
df_return_5y = df_close_5y.pct_change()
df_return_10y = df_close_10y.pct_change()

# check the descriptive statistics of 1 year rate of return
df_return_1y.describe()

,SE,U,NVDA,BILI,FB,DKNG,PDD,GOOG,AMZN,RBLX,NTLA,COIN,UPST,CLPT,S&P500
count,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,147.000000,252.000000,123.000000,203.000000,252.000000,252.000000
mean,0.003245,0.002489,0.001973,0.002735,0.001215,0.000354,0.002039,0.002703,0.000284,0.001427,0.008625,-0.001547,0.015646,0.005817,0.001053
std,0.034946,0.039615,0.025095,0.048456,0.019849,0.034950,0.047780,0.015623,0.016394,0.042439,0.061365,0.035644,0.097194,0.048943,0.008592
min,-0.103195,-0.141312,-0.082178,-0.170397,-0.063099,-0.084906,-0.122561,-0.054630,-0.075649,-0.123584,-0.107134,-0.078664,-0.281373,-0.165829,-0.035288
25%,-0.015823,-0.022938,-0.011579,-0.029657,-0.010505,-0.023178,-0.027351,-0.005245,-0.008416,-0.026959,-0.029680,-0.025041,-0.040730,-0.021639,-0.003424
50%,0.004757,0.002724,0.002308,0.001034,0.000049,-0.001619,-0.002186,0.002564,0.000954,0.000529,0.003227,-0.006277,0.006955,0.000965,0.001211
75%,0.025887,0.027058,0.016599,0.032719,0.013482,0.019694,0.024191,0.009497,0.009655,0.027080,0.035833,0.016360,0.048026,0.031308,0.006560
max,0.108809,0.132524,0.080333,0.221735,0.083227,0.116854,0.222496,0.073961,0.063230,0.213281,0.502083,0.112818,0.893239,0.157664,0.023791


### Rate of Return

In [53]:
# compute the annualized average rate of return across a 1 year, 3 year, 5 year & 10 year horizon.
nday = 252 # number of trading days
df_return_1y_mean = pd.DataFrame(data = df_return_1y.mean(axis = 0), columns = ['1Y Annual Return']) * nday
df_return_3y_mean = pd.DataFrame(data = df_return_3y.mean(axis = 0), columns = ['3Y Annual Return']) * nday
df_return_5y_mean = pd.DataFrame(data = df_return_5y.mean(axis = 0), columns = ['5Y Annual Return']) * nday
df_return_10y_mean = pd.DataFrame(data = df_return_10y.mean(axis = 0), columns = ['10Y Annual Return']) * nday
data_frames = [df_return_1y_mean, df_return_3y_mean, df_return_5y_mean, df_return_10y_mean]
df_return_merge = pd.concat(data_frames, axis = 1)

#df_return_10y_mean.plot.barh()
df_return_merge

,1Y Annual Return,3Y Annual Return,5Y Annual Return,10Y Annual Return
SE,0.817683,1.254634,0.927027,0.927027
U,0.627291,0.869072,0.869072,0.869072
NVDA,0.497248,0.519647,0.627524,0.500265
BILI,0.689307,0.749203,0.735007,0.735007
FB,0.306075,0.317114,0.245016,0.298255
DKNG,0.089136,0.940973,0.940973,0.940973
PDD,0.513900,0.721797,0.649234,0.649234
GOOG,0.681084,0.344006,0.293190,0.271228
AMZN,0.071532,0.246134,0.318702,0.317113
RBLX,0.359675,0.359675,0.359675,0.359675


We see that the annualized returns are skewed to the right due to the broad spike in equity valuations in 2020. Also, some of the companies in the list only went public in the last 5 years, which explains the potentially transitory outperformance against the benchmark. A subtle takeaway is that some of the returns are largely driven by a particular breakout year (e.g. AMZN from March to September 2020), which serves as a timely caution against making large portfolio adjustments based on 1-year performance alone. 

### Variance

In [54]:
# compute the daily standard deviation across a 1 year, 3 year, 5 year & 10 year horizon.
df_std_1y = pd.DataFrame(data = df_close_1y_norm.std(axis = 0), columns = ['1Y Daily Standard Deviation'])  
df_std_3y = pd.DataFrame(data = df_close_3y_norm.std(axis = 0), columns = ['3Y Daily Standard Deviation']) 
df_std_5y = pd.DataFrame(data = df_close_5y_norm.std(axis = 0), columns = ['5Y Daily Standard Deviation']) 
df_std_10y = pd.DataFrame(data = df_close_10y_norm.std(axis = 0), columns = ['10Y Daily Standard Deviation']) 
data_frames = [df_std_1y, df_std_3y, df_std_5y, df_std_10y]
df_std_merge = pd.concat(data_frames, axis = 1)

# compute the daily variance across a 1 year, 3 year, 5 year & 10 year horizon.
df_var_1y = pd.DataFrame(data = df_close_1y_norm.var(axis = 0), columns = ['1Y Daily Variance'])  
df_var_3y = pd.DataFrame(data = df_close_3y_norm.var(axis = 0), columns = ['3Y Daily Variance']) 
df_var_5y = pd.DataFrame(data = df_close_5y_norm.var(axis = 0, skipna = True), columns = ['5Y Daily Variance']) 
df_var_10y = pd.DataFrame(data = df_close_10y_norm.var(axis = 0, skipna = True), columns = ['10Y Daily Variance']) 
data_frames = [df_var_1y, df_var_3y, df_var_5y, df_var_10y]
df_var_merge = pd.concat(data_frames, axis = 1)

df_var_merge

,1Y Daily Variance,3Y Daily Variance,5Y Daily Variance,10Y Daily Variance
SE,0.092519,66.023721,37.497827,37.497827
U,0.058134,0.103720,0.103720,0.103720
NVDA,0.055607,0.705112,9.347320,233.707776
BILI,0.333236,7.769102,10.839026,10.839026
FB,0.022553,0.174231,0.238640,4.927400
DKNG,0.018966,3.640583,3.640583,3.640583
PDD,0.184192,5.392338,3.371982,3.371982
GOOG,0.081840,0.215435,0.440535,4.815670
AMZN,0.002469,0.148504,1.064158,20.106920
RBLX,0.012935,0.012935,0.012935,0.012935


In [55]:
# compute the pearson pairwise correlation matrix
df_return_1y_corr = df_close_1y_norm.corr(method='pearson')
df_return_3y_corr = df_close_3y_norm.corr(method='pearson')
df_return_5y_corr = df_close_5y_norm.corr(method='pearson')
df_return_10y_corr = df_close_10y_norm.corr(method='pearson')

In [56]:
# plotting the 1 year correlation matrix 

sns.set_theme(style="white")
# generate a mask for the upper triangle
mask = np.triu(np.ones_like(df_return_1y_corr, dtype=bool))

# set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# generate a custom diverging colormap
cmap = sns.diverging_palette(240, 5, as_cmap=True)

# draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df_return_1y_corr, mask=mask, cmap=cmap, vmax=1, center=0, vmin = -1,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

This result is sort of expected. In general, we know that the big tech should be highly correlated with the S&P500, and the so-called "high-beta/ARK" stocks should correlate with each other.

In [57]:
# compute the pairwise variance covariance matrix 
df_return_1y_cov = df_close_1y_norm.cov() 
df_return_3y_cov = df_close_3y_norm.cov()
df_return_5y_cov = df_close_5y_norm.cov()
df_return_10y_cov = df_close_10y_norm.cov()

### Risk-Return Measures

In [58]:
# Sharpe ratio 
df_sharpe_merge = np.divide(df_return_merge,df_std_merge)
df_sharpe_merge.rename(columns = {'1Y Annual Return':'1Y Sharpe', '3Y Annual Return':'3Y Sharpe', '5Y Annual Return':'5Y Sharpe','10Y Annual Return':'10Y Sharpe'}, inplace=True)
df_sharpe_merge

,1Y Sharpe,3Y Sharpe,5Y Sharpe,10Y Sharpe
SE,2.688249,0.154407,0.151387,0.151387
U,2.601690,2.698508,2.698508,2.698508
NVDA,2.108670,0.618841,0.205252,0.032724
BILI,1.194088,0.268791,0.223253,0.223253
FB,2.038087,0.759719,0.501559,0.134363
DKNG,0.647233,0.493164,0.493164,0.493164
PDD,1.197412,0.310833,0.353556,0.353556
GOOG,2.380774,0.741155,0.441732,0.123596
AMZN,1.439496,0.638707,0.308945,0.070720
RBLX,3.162526,3.162526,3.162526,3.162526


Comparing the sharpe ratios of the individual stocks against the benchmark, it becomes apparent that beating the market on a risk-adjusted basis is no simple feat (at least when we use volatility as a risk measure). The sharpe ratio of individual names might be higher than the S&P500 on their peak years, but they tend to fall behind on historical aggregates of longer time horizons.

## 2. Portfolio Optimization

In this section, we will start off by experimenting with the PyPortfolioOpt library (Martin, 2021). His work can be accessed [here](https://joss.theoj.org/papers/10.21105/joss.03066).

In [59]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.hierarchical_portfolio import HRPOpt

### Current Allocation

In [62]:
# current weightage 
arr_w = np.array([[0.40,0.12,0.10,0.06,0.06,0.06,0.04,0.04,0.04,0.04,0.01, 0.01, 0.01, 0.01]])
df_w = pd.DataFrame(data = arr_w, columns = s_order)
df_w = df_w.rename(index={0: "weight"})
df_w

,SE,U,NVDA,BILI,FB,DKNG,PDD,GOOG,AMZN,RBLX,NTLA,COIN,UPST,CLPT
weight,0.4,0.12,0.1,0.06,0.06,0.06,0.04,0.04,0.04,0.04,0.01,0.01,0.01,0.01


In [63]:
arr_w.sum()

1.0000000000000002

In [64]:
# split dataframe into portfolio and benchmark
df_return_1y_mean_port = df_return_1y_mean.iloc[0:n-1,:]
df_return_1y_mean_bench = df_return_1y_mean.loc['S&P500']

df_return_1y_cov_port = df_return_1y_cov.iloc[0:n-1,0:n-1]

In [65]:
# convert dataframe back to numpy array for ease of manipulation
np_return_1y_mean = df_return_1y_mean_port.T.to_numpy()

np_return_1y_cov = df_return_1y_cov_port.to_numpy()

In [66]:
# portfolio return 
portfolio_return_1y = np.inner(np_return_1y_mean,arr_w)
portfolio_var_1y = np.matmul(np.matmul(arr_w,np_return_1y_cov),arr_w.T)
portfolio_sharpe_1y = portfolio_return_1y/np.sqrt(portfolio_var_1y)
benchmark_sharpe_1y = df_sharpe_merge['1Y Sharpe']['S&P500']
print(f'Portfolio 1y return:{portfolio_return_1y}; Portfolio 1y variance:{portfolio_var_1y}')
print(f'Portfolio 1y sharpe ratio:{portfolio_sharpe_1y}; S&P500 1y Sharpe: {benchmark_sharpe_1y}')

Portfolio 1y return:[[0.65411538]]; Portfolio 1y variance:[[0.04685541]]
Portfolio 1y sharpe ratio:[[3.02186119]]; S&P500 1y Sharpe: 2.7168339607501597


Here, we see the diversification effects of a multi-asset portfolio. While the 1 year portfolio returns remains at a respectable 68%, the portfolio volatility is drastically reduced as compared to the volatility of the individual stocks. In MPT terminology, the idiosyncratic risks of individual assets is reduced.

However, we once again see the power of the S&P500, which boosts a Sharpe ratio of 2.60 compared to our current portfolio Sharpe ratio of 2.87. Now let's see if we can use the MPT techniques to construct a portfolio that outperforms the market on a risk-adjusted basis. 

### Maximum Sharpe Portfolio 

In [67]:
# split dataframe into portfolio and benchmark
df_close_1y_port = df_close_1y.iloc[:,0:n-1]
df_close_1y_bench = df_close_1y['S&P500']

In [68]:
# Expected returns
mu_a = expected_returns.mean_historical_return(df_close_1y_port, compounding = False)
mu_g = expected_returns.mean_historical_return(df_close_1y_port, compounding = True)
mu_ema = expected_returns.ema_historical_return(df_close_1y_port, compounding=True)
mu_capm = expected_returns.capm_return(df_close_1y_port)
data_frames = [mu_a,mu_g,mu_ema,mu_capm]
df_merge = pd.concat(data_frames, axis = 1)
df_merge = df_merge.rename(columns = {0: 'Arithmetic', 1: 'Geometric', 2:'Exponential MA', 'mkt': 'CAPM'})
df_merge

,Arithmetic,Geometric,2021-10-07 00:00:00,CAPM
SE,0.817683,0.940569,1.165147,1.411667
U,0.627291,0.535928,0.829344,1.496634
NVDA,0.497248,0.518634,0.759793,0.886220
BILI,0.689307,0.488357,0.307971,1.879026
FB,0.306075,0.292635,0.284259,0.557975
DKNG,0.089136,-0.061035,0.050584,1.055284
PDD,0.513900,0.266176,0.174816,1.774725
GOOG,0.681084,0.915133,0.817745,0.392644
AMZN,0.071532,0.038414,0.057604,0.490444
RBLX,0.359675,0.148600,0.278265,1.199162


In [72]:
# Risk matrix
S = risk_models.sample_cov(df_close_1y_port)

In [73]:
# Optimize for maximum Sharpe ratio using arithmetic mean & sample covariance matrix
ef_a = EfficientFrontier(mu_a, S)
raw_weights = ef_a.max_sharpe()
cleaned_weights = ef_a.clean_weights()
# ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef_a.portfolio_performance(verbose=True)

OrderedDict([('SE', 0.0), ('U', 0.0), ('NVDA', 0.0), ('BILI', 0.0), ('FB', 0.0), ('DKNG', 0.0), ('PDD', 0.0), ('GOOG', 0.7178), ('AMZN', 0.0), ('RBLX', 0.0), ('NTLA', 0.10102), ('COIN', 0.0), ('UPST', 0.11051), ('CLPT', 0.07067)])
Expected annual return: 124.8%
Annual volatility: 30.1%
Sharpe Ratio: 4.08


(1.2477672937898914, 0.3008944483617685, 4.080391979561331)

Here, we see the central limitation behind the methodology of using historical returns to estimate the expected (future) returns. The allocation model placed a disproportionate weight on the outperformers in the past year (i.e. google, upstart & clearpoint), outputing a portfolio that looks good on paper but probably does not hold its weight over time. Let's try to do better. 

In [75]:
# Optimize for maximum Sharpe ratio using capm returns & sample covariance matrix
ef_capm = EfficientFrontier(mu_capm, S)
raw_weights = ef_capm.max_sharpe()
cleaned_weights = ef_capm.clean_weights()
print(cleaned_weights)
ef_capm.portfolio_performance(verbose=True)

OrderedDict([('SE', 0.07483), ('U', 0.06961), ('NVDA', 0.0567), ('BILI', 0.06841), ('FB', 0.05982), ('DKNG', 0.06509), ('PDD', 0.07152), ('GOOG', 0.08584), ('AMZN', 0.11233), ('RBLX', 0.0589), ('NTLA', 0.0639), ('COIN', 0.06466), ('UPST', 0.07234), ('CLPT', 0.07604)])
Expected annual return: 126.1%
Annual volatility: 34.5%
Sharpe Ratio: 3.59


(1.2606687354057895, 0.34539021021503863, 3.592078462887972)

Interestingly, we see that using $\mu_{CAPM}$ gives us a portfolio allocation that is almost equal weighting across all assets. However, it's absolute and risk-adjusted returns are both inferior compared to the arithmetic mean portfolio. 

### Black-Litterman Portfolio

In [77]:
S = risk_models.sample_cov(df_close_1y_port)
view_dict = {"SE": 0.50, "NVDA": 0.40, "U": 0.40, "BILI": 0.39, "PDD": 0.37,  "DKNG": 0.35, "FB": 0.33, "GOOG": 0.31, "AMZN": 0.31, "RBLX": 0.27, }
confidence_dict = np.array([0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8])
bl = BlackLittermanModel(S, pi="equal", absolute_views=view_dict, omega="idzorek", view_confidences = confidence_dict,tau=0.1)
rets = bl.bl_returns()
var = pd.DataFrame(np.diag(S), index= S.index , columns = ['Variance'], dtype = float)

# black litterman returns
df_merge = pd.concat([rets, var], axis = 1)
df_merge = df_merge.rename(columns = {0: 'Return'})
df_merge['Sharpe'] = df_merge['Return']/df_merge['Variance']
df_merge

,Return,Variance,Sharpe
SE,0.458889,0.307752,1.491097
U,0.389029,0.395474,0.983704
NVDA,0.383242,0.158702,2.414856
BILI,0.384095,0.591699,0.649140
FB,0.317876,0.099280,3.201825
DKNG,0.333847,0.307817,1.084564
PDD,0.393322,0.575306,0.683675
GOOG,0.283514,0.061506,4.609534
AMZN,0.292664,0.067730,4.321019
RBLX,0.275372,0.453866,0.606726


In [78]:
ef_bl = EfficientFrontier(rets, S)
raw_weights = ef_bl.max_sharpe()
cleaned_weights = ef_bl.clean_weights()
print(cleaned_weights)
ef_bl.portfolio_performance(verbose=True)

OrderedDict([('SE', 0.07473), ('U', 0.01518), ('NVDA', 0.067), ('BILI', 0.0), ('FB', 0.07395), ('DKNG', 0.04234), ('PDD', 0.0), ('GOOG', 0.41626), ('AMZN', 0.27814), ('RBLX', 0.0), ('NTLA', 0.0), ('COIN', 0.02191), ('UPST', 0.00466), ('CLPT', 0.00583)])
Expected annual return: 30.9%
Annual volatility: 23.0%
Sharpe Ratio: 1.26


(0.30907709544059847, 0.2295512441287917, 1.2593139999642489)

We see that the model still favors stocks with high sharpe-ratio (e.g. GOOG & AMZN).

Now let's try out recent developments in the field like shrinkage and Hierarchical Risk Parity (López de Prado, 2016), along with some novel experimental features like exponentially-weighted covariance matrices.

In [ ]:
hrp = HRPOpt()

## 3.Extensions

As per PyPortfolioOpt's roadmap, I aim to include conditional drawdown optimization (Chekhlov et al., 2005), risk parity portfolios (Spinu,2013), higher moment optimization (Harvey et al., 2010), and factor models into my analysis in the near future. Stay tuned!